In [1]:
import sys
import multiprocessing as mp
import threading
import time
import random

In [2]:
class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1

In [15]:
class Puente():
  def __init__(self):    
    self.lock = threading.Condition()
    self.number = 0

  def entrar(self, sentido,indice):         # used for synchronization
    with self.lock:
      while self.number > 3:
        sys.stdout.write("-------------Carro %s en sentido %s esperando por el puente----------\n" % (indice,sentido))
        self.lock.wait()
      self.number += 1
      sys.stdout.write("--------Carro en %s sentido %s pasando por el puente-----\n"%(indice,sentido))
      self.lock.notifyAll()

  def salir(self, sentido,indice):         # used for synchronization
    with self.lock:
      self.number -= 1
      sys.stdout.write("--------Carro en %s sentido %s ya pasó por el puente-----\n"%(indice,sentido))
      self.lock.notifyAll()

In [16]:
class Carro(threading.Thread):
    def __init__(self,semaforo,puente,indice):
        threading.Thread.__init__(self)
        self.dir = random.randint(0,1)
        self.semaforo = semaforo
        self.puente = puente
        self.indice = indice

    def run(self):
        while True:
          if self.dir == 0:
            sys.stdout.write("Carro %s en sentido izquierdo (0) quiere pasar\n" %(self.indice))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)
            self.puente.entrar(self.dir,self.indice)        
            time.sleep(1)
            self.puente.salir(self.dir,self.indice)  
            time.sleep(1)
            self.semaforo.up()             
            time.sleep(1)

          if self.dir == 1:
            sys.stdout.write("Carro %s en sentido derecho (1) quiere pasar\n"%(self.indice))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)
            self.puente.entrar(self.dir,self.indice)        
            time.sleep(1)
            self.puente.salir(self.dir,self.indice)  
            time.sleep(1)
            self.semaforo.up()             
            time.sleep(1)

In [ ]:
if __name__ == '__main__':
    n_izq = 5
    c = [Puente() for i in range(n_izq)]
    semaforo = Semaphore(2)
    sys.stdout.write("\n")

    p_izq = [ Carro(semaforo,c[i],i) for i in range(n_izq) ]
    
    for i in range(n_izq):
        p_izq[i].start()
        
    for i in range(n_izq):
        p_izq[i].join()